In [ ]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

%load_ext autoreload
%autoreload 2

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
import plotly_express as px

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

import sys
sys.path.append('../src')
import funciones as f
import soporte_prep as sp

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extracción de tabla principal

In [ ]:
driver = webdriver.Chrome()
url_accidentes = "https://www.one.gob.do/datos-y-estadisticas/"
driver.get(url_accidentes)
driver.maximize_window()

sleep(1)  # Esperar un segundo entre cada scroll

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/ul/li[2]/a').click()
except:
    print('No encuentro el botón')

sleep(2)

for i in range(2):
    try:
        driver.execute_script("window.scrollBy(0, 500);")  # Desplazarse 1000 píxeles hacia abajo
    except:
        print('No he podido descender')

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/header/div[2]').click()
except:
    print('No encuentro el botón para ampliar')
sleep(2)
try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/header/div[1]').click()
except:
    print('No encuentro el botón para abrir pestañas')
sleep(2)
try:
    driver.execute_script("window.scrollBy(0, 700);")
except:
    print('No he podido descender')

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div/div/div/div[2]/div/table/tbody/tr[1]/td/a').click()
except:
    print('No encuentro el botón para descargar') #descargo el libro de códigos de accidentes de tráfico

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div/div/div/div[2]/div/table/tbody/tr[8]/td/a').click()
except:
    print('No encuentro el botón para descargar') #descargo los accidentes de tráfico

    

## Tablas secundarias

In [ ]:
driver = webdriver.Chrome()
url_accidentes = "file:///C:/Users/DELL/Downloads/Libro%20de%20c%C3%B3digo%20accidentes%20de%20tr%C3%A1nsito%202007-2023%20(1).htm"
driver.get(url_accidentes)
driver.maximize_window()

sleep(1)  # Esperar un segundo entre cada scroll

sopa_accidentes = BeautifulSoup(driver.page_source)
lista_tablas = sopa_accidentes.findAll('div', {'class': 'itemBody'})
lista_tablas[0]

In [ ]:
dataframes = []

for i, tabla in enumerate(lista_tablas):
    rows = tabla.find_all('tr')

    data = []
    for row in rows:
        cells = row.find_all('td')
        data.append([cell.get_text(strip=True) for cell in cells if cell.get_text(strip=True)])

    df = pd.DataFrame(data)

    dataframes.append(df)

print(f"Se generaron {len(dataframes)} DataFrames.")

In [ ]:
df_diasem = dataframes[0]
df_mes = dataframes[1]
df_rhora = dataframes[3]
df_region = dataframes[4]
df_provincia = dataframes[5]
df_tipovia = dataframes[6]
df_accidente = dataframes[7]
df_mediot = dataframes[9]
df_choque = dataframes[10]

In [ ]:
dataframes_list = [
    df_region, df_provincia, df_diasem, df_mes, df_rhora,
    df_tipovia, df_accidente,
    df_mediot, df_choque
]

for i, df in enumerate(dataframes_list):
    # Recortar las filas a partir de la fila 3
    df = df.iloc[2:]
    
    # Mover el valor de la columna 2 a la columna 1 en la primera fila
    df.iloc[0, 1] = df.iloc[0, 2]
    df.iloc[0,0] = 1
    
    # Seleccionar solo la columna 1
    df = df[[0,1]]
    
    # Reiniciar el índice
    df.reset_index(drop=True, inplace=True)
    df = df[:-1]
    df.columns = ['Valor', 'Significado']
    df['Valor'] = df['Valor'].astype('int64')
    # Guardar el DataFrame transformado de nuevo en la lista
    dataframes_list[i] = df

# Asignar los DataFrames de vuelta a sus variables
df_region, df_provincia, df_diasem, df_mes, df_rhora, df_tipovia, df_accidente, df_mediot, df_choque = dataframes_list

In [ ]:
dataframes = [
    (df_diasem, 'df_diasem.csv'),
    (df_mes, 'df_mes.csv'),
    (df_rhora, 'df_rhora.csv'),
    (df_region, 'df_region.csv'),
    (df_provincia, 'df_provincia.csv'),
    (df_tipovia, 'df_tipovia.csv'),
    (df_accidente, 'df_accidente.csv'),
    (df_mediot, 'df_mediot.csv'),
    (df_choque, 'df_choque.csv')
]

for df, filename in dataframes:
    df.to_csv(f'../Datos/datos_accidentes/{filename}')

In [ ]:
df_diasem.to_csv('../Datos/datos_accidentes/df_diasem.csv')
df_mes.to_csv('../Datos/datos_accidentes/df_mes.csv')
df_rhora.to_csv('../Datos/datos_accidentes/df_rhora.csv')
df_region.to_csv('../Datos/datos_accidentes/df_region.csv')
df_provincia.to_csv('../Datos/datos_accidentes/df_provincia.csv')
df_tipovia.to_csv('../Datos/datos_accidentes/df_tipovia.csv')
df_accidente.to_csv('../Datos/datos_accidentes/df_accidente.csv')
df_mediot.to_csv('../Datos/datos_accidentes/df_mediot.csv')
df_choque.to_csv('../Datos/datos_accidentes/df_choque.csv')